    # Run & Deploy Streamlit UI

This notebook will **create the corrected Streamlit app** (`app.py`) inside the project and provide the command to **launch it in Kaggle Web App mode** or locally.

**How to use:**
1. Run the first code cell to write `app.py` and `requirements.txt` into `/mnt/data/capstone_project/ui/`.
2. If you're on Kaggle, run the final cell to start the Streamlit server in Web App mode.
3. If running locally, run `streamlit run /path/to/capstone_project/ui/app.py`.


In [1]:
# This cell writes the Streamlit app file into the project.
app_code = r'''
import streamlit as st
import os, sys, time, json

ROOT = r'C:\Users\HEALTHY MACHINES\OneDrive\Desktop\capstone_project_package'

if ROOT not in sys.path:
    sys.path.insert(0, ROOT)
    sys.path.insert(0, os.path.join(ROOT, "src"))

card_image_path = os.path.join(ROOT, "A_pair_of_digital_graphics_features_Faiq_Ahamed’s_.png")

from orchestrator import Orchestrator
from agents import ResearchAgent, SummarizerAgent, CriticAgent, WriterAgent
from tool_adapter import Tool, simple_search
from a2a_simulator import A2ABus
from memory import Session, MemoryStore
from observability import log_event, trace_span, emit_metric

st.set_page_config(page_title="AI Research Team - Demo", layout="wide")

# Header with image and title
col1, col2 = st.columns([1,3])
with col1:
    try:
        st.image(card_image_path, width=220)
    except Exception:
        st.write("Project image not available")
with col2:
    st.title("AI Research Team — Multi-Agent Research Automation")
    st.markdown("**Author:** Faiq Ahamed  •  **Track:** Freestyle")

# Sidebar
st.sidebar.header("Settings & Mode")
USE_MOCK = st.sidebar.checkbox("Use Mock Mode (recommended)", value=True)
st.sidebar.markdown("If unchecked, the app will attempt to use real API credentials from environment variables.")
if not USE_MOCK:
    st.sidebar.text("Ensure GOOGLE_API_KEY and GOOGLE_APPLICATION_CREDENTIALS are set.")

st.sidebar.markdown("---")
show_logs = st.sidebar.checkbox("Show Logs/Traces", value=False)
show_memory = st.sidebar.checkbox("Show Memory Store", value=False)

# Main
st.subheader("Run Research Pipeline")
query = st.text_input("Enter research query", value="Recent breakthroughs in quantum computing and impact on AI (2024-2025)")
run_button = st.button("Run Research")
clear_button = st.button("Clear Outputs")

# Initialize agents and orchestrator
search_tool = Tool("web_search", simple_search)
research = ResearchAgent("ResearchAgent", tools={"search": search_tool}, use_mock=USE_MOCK)
summarizer = SummarizerAgent("SummarizerAgent", use_mock=USE_MOCK)
critic = CriticAgent("CriticAgent", use_mock=USE_MOCK)
writer = WriterAgent("WriterAgent", use_mock=USE_MOCK)
bus = A2ABus()
orch = Orchestrator(
    agents=[research, summarizer, critic, writer],
    bus=bus,
    memory_path=str(ROOT + "/data/processed/memory_store.json"),
    use_mock=USE_MOCK
)

memory = MemoryStore(str(ROOT + "/data/processed/memory_store.json"))

progress_bar = st.progress(0)
status_text = st.empty()

findings_expander = st.expander("Research Findings", expanded=True)
summary_expander = st.expander("Summarizer Output", expanded=True)
critique_expander = st.expander("Critic Output", expanded=True)
draft_expander = st.expander("Final Draft", expanded=True)

# Optional upload
st.sidebar.header("Upload Document (optional)")
uploaded_file = st.sidebar.file_uploader("Upload PDF/TXT to include in research", type=['pdf','txt'])

if run_button:
    status_text.info("Starting pipeline...")
    session = Session(f"ui-session-{int(time.time())}")
    session.add_turn("user", query)
    steps = ["research", "summarize", "critique", "write", "finalize"]
    results = {}

    for i, step in enumerate(steps):
        status_text.info(f"Running step: {step}")
        time.sleep(0.5)
        progress = int(((i+1)/len(steps)) * 100)
        progress_bar.progress(progress)

        if step == "finalize":
            res = orch.run_pipeline(session_id=session.session_id, user_query=query)
            results = res

    progress_bar.progress(100)
    status_text.success("Pipeline complete.")

    with findings_expander:
        st.write("**Findings (raw):**")
        st.text_area("Findings", value=results.get("summary","No findings produced"), height=180)

    with summary_expander:
        st.write("**Summary:**")
        st.text_area("Summary", value=results.get("summary","No summary produced"), height=160)

    with critique_expander:
        st.write("**Critique:**")
        st.text_area("Critique", value=results.get("critique","No critique produced"), height=140)

    with draft_expander:
        st.write("**Final Draft:**")
        st.text_area("Draft", value=results.get("final_draft","No draft produced"), height=300)
        st.download_button(
            "Download Draft as TXT",
            results.get("final_draft",""),
            file_name="final_draft.txt"
        )

    if show_logs:
        st.subheader("Observability: Logs & Metrics")
        try:
            with open(ROOT + "/data/processed/agent_logs.jsonl") as f:
                lines = f.read().strip().split("\\n")[-20:]
            st.write(lines)
        except Exception:
            st.write("No logs found.")

        try:
            with open(ROOT + "/data/processed/agent_metrics.json") as f:
                st.json(json.load(f))
        except Exception:
            st.write("No metrics found.")

    if show_memory:
        st.subheader("Memory Store")
        try:
            st.json(memory.store)
        except Exception:
            st.write("No memory file found.")

if clear_button:
    st.experimental_rerun()

st.sidebar.markdown("---")
st.sidebar.write("© Faiq Ahamed — AI Research Team Capstone")
'''

ui_dir = r'C:\Users\HEALTHY MACHINES\OneDrive\Desktop\capstone_project_package\ui'
import os
os.makedirs(ui_dir, exist_ok=True)
with open(os.path.join(ui_dir, "app.py"), "w", encoding="utf-8") as f:
    f.write(app_code)
print("Wrote app.py to:", os.path.join(ui_dir, "app.py"))

# Write requirements
reqs = "streamlit\nnumpy\npandas\n"
with open(os.path.join(ui_dir, "requirements.txt"), "w", encoding="utf-8") as f:
    f.write(reqs)
print("Wrote requirements.txt to:", os.path.join(ui_dir, "requirements.txt"))

# Confirm card image exists
card_img = "A_pair_of_digital_graphics_features_Faiq_Ahamed’s_.png"
print("Card image path:", card_img)
print("Exists:", os.path.exists(card_img))


Wrote app.py to: C:\Users\HEALTHY MACHINES\OneDrive\Desktop\capstone_project_package\ui\app.py
Wrote requirements.txt to: C:\Users\HEALTHY MACHINES\OneDrive\Desktop\capstone_project_package\ui\requirements.txt
Card image path: A_pair_of_digital_graphics_features_Faiq_Ahamed’s_.png
Exists: False


## Launching the Streamlit App

### On Kaggle (Web App mode)
Run the command in a code cell below. Make sure the notebook has the project files under `/kaggle/working/capstone_project`.

```
!streamlit run /kaggle/working/capstone_project/ui/app.py --server.port $PORT --server.address 0.0.0.0 &
```

After running, click the **Open in new tab** link produced by the cell.

### Locally
From your project root, run:
```
streamlit run /path/to/capstone_project/ui/app.py
```



In [2]:
# Kaggle-friendly launcher (run this cell on Kaggle to start the Streamlit app in Web App mode)
# Note: on local machines, use the CLI `streamlit run` command instead.
import os
if os.path.exists("/kaggle/working"):
    print("Starting Streamlit in Kaggle Web App mode...")
    get_ipython().system('streamlit run /kaggle/working/capstone_project/ui/app.py --server.port $PORT --server.address 0.0.0.0 &')
else:
    print("Not on Kaggle. To run locally, use: streamlit run /path/to/capstone_project/ui/app.py")

Not on Kaggle. To run locally, use: streamlit run /path/to/capstone_project/ui/app.py
